# 01 - Data Ingestion Pipeline

This notebook demonstrates the ingestion pipeline for the **Retail Multi-Brand Data Project**.  
We ingest raw CSV extracts (provided as a zipped dataset) into **DuckDB**, and export them into **Parquet** format for efficient downstream analytics.

**Steps in this notebook:**
1. Setup environment & paths  
2. Mount data source  
3. Extract raw CSV files  
4. Load CSVs into **DuckDB tables**  
5. Export tables into **Parquet (staging layer)**  
6. Verify row counts & samples  

In [ ]:
# ==========================
# STEP 0: Install libraries
# ==========================
!pip install duckdb pandas pyarrow

# ==========================
# STEP 1: Mount Google Drive
# ==========================
from google.colab import drive
drive.mount('/content/drive')

import os, zipfile, duckdb

# ==========================
# STEP 2: Define Paths
# ==========================
ZIP_PATH = '/content/drive/My Drive/Dataset/Retail Multi-Brand Datasets.zip'
EXTRACT_DIR = '/content/drive/My Drive/Dataset/Retail_Multibrand_extracted'
DUCKDB_PATH = '/content/drive/My Drive/Dataset/Retail_Multibrand_extracted/multibrand_retail.duckdb'
EXPORT_DIR = '/content/drive/My Drive/Dataset/Retail_Multibrand_extracted/Parquet'

# ==========================
# STEP 3: Extract ZIP
# ==========================
if not os.path.exists(EXTRACT_DIR):
    os.makedirs(EXTRACT_DIR)

with zipfile.ZipFile(ZIP_PATH, 'r') as z:
    z.extractall(EXTRACT_DIR)

print("Extracted files:", os.listdir(EXTRACT_DIR))

# ==========================
# STEP 4: Load into DuckDB
# ==========================
con = duckdb.connect(DUCKDB_PATH)

file_map = {
    "brands.csv": "brands",
    "customers.csv": "customers",
    "final_inventory.csv": "inventory",
    "online_channels.csv": "channels",
    "products.csv": "products",
    "promotions.csv": "promotions",
    "sales_transactions_simulated.csv": "sales",
    "stores_enhanced.csv": "stores"
}

for fname, tname in file_map.items():
    fpath = os.path.join(EXTRACT_DIR, fname)
    print(f"Loading {fname} -> {tname}")
    con.execute(f"CREATE OR REPLACE TABLE {tname} AS SELECT * FROM read_csv_auto('{fpath}')")
    cnt = con.execute(f"SELECT COUNT(*) FROM {tname}").fetchone()[0]
    print(f" -> {tname}: {cnt} rows")

# ==========================
# STEP 5: Export to Parquet
# ==========================
if not os.path.exists(EXPORT_DIR):
    os.makedirs(EXPORT_DIR)

for fname, tname in file_map.items():
    out_path = os.path.join(EXPORT_DIR, f"{tname}.parquet")
    print(f"Exporting {tname} -> {out_path}")
    con.execute(f"COPY {tname} TO '{out_path}' (FORMAT PARQUET)")

# ==========================
# STEP 6: Verification
# ==========================
print("\nTables in DuckDB:")
print(con.execute("SHOW TABLES").fetchdf())

print("\nSample from sales:")
print(con.execute("SELECT * FROM sales LIMIT 5").fetchdf())

print("\nParquet files created:", os.listdir(EXPORT_DIR))

con.close()
